# Notebook for manually calculate measurements and make CIT specimen file entries based on .UP files

The issue is during an AF run we did not tick off down direction measurements. So we have to manually calculate the up directions and make CIT specimen file entries based on all the .UP files with the specimen measurements. 

In [22]:
import pmagpy.pmag as pmag
import pmagpy.ipmag as ipmag
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from IPython.display import display
%matplotlib inline

In [23]:
steps = ['NRM   ', 'AF  10', 'AF  20', 'AF  30', 'AF  40', 'AF  50', 
                 'AF  60', 'AF  70', 'AF  80', 'AF  90', 'AF 100', 
                 'AF 110', 'AF 120', 'AF 130', 'AF 140', 'AF 150', 
                 'AF 160', 'AF 170', 'AF 180', 'AF 190', 'AF 200', 
                 'AF 210', 'AF 220', 'AF 230', 'AF 240', 'AF 250',
                 'AF 260', 'AF 270', 'AF 280', 'AF 290', 'AF 300',
                 'AF 310', 'AF 320', 'AF 330', 'AF 340', 'AF 350',
                 'AF 360', 'AF 370', 'AF 380', 'AF 390', 'AF 400',
                 'AF 410', 'AF 420', 'AF 430', 'AF 440', 'AF 450',
                 'AF 460', 'AF 470', 'AF 480', 'AF 490', 'AF 500']

test_sample = 'F1917.5_8'
real_sample = 'F1917.0_15'


In [90]:
# in each .UP file for each sample, the first two rows are zero baselines, 
# the next four rows are the sample measurements (not rotated to uniform coordinates), 
# and the last four rows are the four holder measurements
# we need to subtract the zero baselines and the average holder measurements 
# from the sample averages to generate CIT specimen data files that match 
# the magnetometer computer output

all_spec_decs = [] # all specimen coordinate decs
all_spec_incs = [] # all specimen coordinate incs
all_spec_moms = [] # all specimen coordinate moments

for step in steps:
    # read the .UP file
    this_UP_file =  pd.read_table('./F1917/'+step+'.UP', sep=',|\|', header=None, skiprows=1)
    
    # filter for the entries for this sample
    target_sample_entries = this_UP_file[(this_UP_file[0] == real_sample)]

    # get the zero baselines
    this_baseline_x, this_baseline_y, this_baseline_z = target_sample_entries[(target_sample_entries[3]=='Z')][[6,7,8]].mean()
    # print(this_baseline_x, this_baseline_y, this_baseline_z)
    # get the average holder measurements
    this_holder = target_sample_entries[(target_sample_entries[3]=='H')][[6,7,8]].reset_index(drop=True)
    this_holder_average_x = (this_holder[6].iloc[0]+this_holder[7].iloc[1]-this_holder[6].iloc[2]-this_holder[7].iloc[3])/4
    this_holder_average_y = (this_holder[7].iloc[0]-this_holder[6].iloc[1]-this_holder[7].iloc[2]+this_holder[6].iloc[3])/4
    this_holder_average_z = (this_holder[8].iloc[0]+this_holder[8].iloc[1]+this_holder[8].iloc[2]+this_holder[8].iloc[3])/4

    # get the sample measurements
    this_sample = target_sample_entries[(target_sample_entries[3]=='S')][[0, 6,7,8]].reset_index(drop=True)
    this_sample[6] = this_sample[6] - this_baseline_x
    this_sample[7] = this_sample[7] - this_baseline_y
    this_sample[8] = this_sample[8] - this_baseline_z
    this_sample_average_x = (this_sample[6].iloc[0]+this_sample[7].iloc[1]-this_sample[6].iloc[2]-this_sample[7].iloc[3])/4
    this_sample_average_y = (this_sample[7].iloc[0]-this_sample[6].iloc[1]-this_sample[7].iloc[2]+this_sample[6].iloc[3])/4
    this_sample_average_z = (this_sample[8].iloc[0]+this_sample[8].iloc[1]+this_sample[8].iloc[2]+this_sample[8].iloc[3])/4

    # display(this_sample)
    # subtract the baselines and holder averages from the sample averages
    this_sample_average_x = this_sample_average_x - this_holder_average_x
    this_sample_average_y = this_sample_average_y - this_holder_average_y
    this_sample_average_z = this_sample_average_z - this_holder_average_z


    # target_sample_entries = this_UP_file[(this_UP_file[0] == test_sample) & (this_UP_file[3]=='S')]
    this_step_spec_dec, this_step_spec_inc, this_step_spec_mom = pmag.cart2dir([this_sample_average_x, this_sample_average_y, this_sample_average_z])
    this_step_spec_dec = this_step_spec_dec
    this_step_spec_mom = this_step_spec_mom*1e-5

    # save the dec inc and moment to the lists
    all_spec_decs.append(this_step_spec_dec)
    all_spec_incs.append(this_step_spec_inc)
    all_spec_moms.append(this_step_spec_mom)

# open the target sample file that is empty
target_sample_file = open('./F1917/'+real_sample, 'r')
target_sample_file_lines = target_sample_file.readlines()

# get the first title line
target_sample_title_line = target_sample_file_lines[0]

# get the strat height, core strike and dip, and bedding strike and dip
target_sample_orientation_line = target_sample_file_lines[1]
target_sample_strat_height = target_sample_orientation_line[0:7]
target_sample_core_strike = target_sample_orientation_line[8:13]
target_sample_core_dip = target_sample_orientation_line[14:19]
target_sample_bedding_strike = target_sample_orientation_line[20:25]
target_sample_bedding_dip = target_sample_orientation_line[26:31]
targe_sample_orientations_line_rest = target_sample_orientation_line[32:]

# print((target_sample_orientation_line))
# print(target_sample_strat_height) 
# print(target_sample_core_strike)
# print(target_sample_core_dip)
# print(target_sample_bedding_strike)
# print(target_sample_bedding_dip)
# print(targe_sample_orientations_line_rest)


# calcualte dec, inc in geographic coordinates
all_spec_decs_geo, all_spec_incs_geo = pmag.dotilt_V(np.array([all_spec_decs, all_spec_incs, 
                                                               np.full(len(all_spec_decs), float(target_sample_core_strike)+90), 
                                                               np.full(len(all_spec_decs), float(target_sample_core_dip))]).T)

all_spec_decs_tilt, all_spec_incs_tilt = pmag.dotilt_V(np.array([all_spec_decs_geo, all_spec_incs_geo,
                                                                np.full(len(all_spec_decs), float(target_sample_bedding_strike)+90),
                                                                np.full(len(all_spec_decs), float(target_sample_bedding_dip))]).T)
# all_spec_decs_tilt, all_spec_incs_tilt

CIT_lines = []
CIT_lines.append(target_sample_title_line)
CIT_lines.append(target_sample_orientation_line)

for step, i in zip(steps, range(len(all_spec_decs))):

    this_line = step + ' {:05.1f}'.format(all_spec_decs_geo[i]) +\
                       ' {:05.1f}'.format(all_spec_incs_geo[i]) +\
                       ' {:05.1f}'.format(all_spec_decs_tilt[i]) +\
                       ' {:05.1f}'.format(all_spec_incs_tilt[i]) +\
                       ' {:8.2E}'.format(all_spec_moms[i]) +\
                       ' 000.1'+\
                       ' {:05.1f}'.format(all_spec_decs[i]) +\
                       ' {:05.1f}'.format(all_spec_incs[i]) +\
                       ' 0.000001 0.000001 0.000001 hargrave 2023-04-09 12:00:00\n'
    # print(this_line)
    CIT_lines.append(this_line)


# open up a new file to write in the new lines
new_file = open('./F1917/'+real_sample+'new', 'w')
for line in CIT_lines:
    new_file.write(line)
new_file.close()